In [1]:
import torch
import torchvision
import torch.nn as nn
import torchvision.transforms as transforms

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

num_epochs = 5
num_classes = 10
batch_size = 100
learning_rate = 0.001

train_dataset = torchvision.datasets.MNIST(root="../data/",
                                           train=True,
                                           transform=transforms.ToTensor(),
                                           download=True)

test_dataset = torchvision.datasets.MNIST(root="../data",
                                          train=False,
                                          transform=transforms.ToTensor())

train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=batch_size,
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=batch_size,
                                          shuffle=False)

# 두개의 Convolutional layer를 갖는(convolution 연산과 pooling 연산을 하는) CNN 정의
class ConvNet(nn.Module):
    def __init__(self, num_classes=10):
        super(ConvNet, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(1, 16, kernel_size=5, stride=1, padding=2),
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        self.layer2 = nn.Sequential(
            nn.Conv2d(16, 32, kernel_size=5, stride=1, padding=2),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        self.fc = nn.Linear(7*7*32, num_classes)
        
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.reshape(out.size(0), -1)
        out = self.fc(out)
        return out
    
model = ConvNet(num_classes).to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

total_step = len(train_loader)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)
        
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i+1) % 100 == 0:
            print(f"Epoch [{epoch+1/num_epochs}], Step [{(i+1)/total_step}], Loss: {loss.item():.4f}")
    

Epoch [0.2], Step [0.16666666666666666], Loss: 0.1634
Epoch [0.2], Step [0.3333333333333333], Loss: 0.0963
Epoch [0.2], Step [0.5], Loss: 0.0605
Epoch [0.2], Step [0.6666666666666666], Loss: 0.0690
Epoch [0.2], Step [0.8333333333333334], Loss: 0.1093
Epoch [0.2], Step [1.0], Loss: 0.1030
Epoch [1.2], Step [0.16666666666666666], Loss: 0.0280
Epoch [1.2], Step [0.3333333333333333], Loss: 0.0362
Epoch [1.2], Step [0.5], Loss: 0.0842
Epoch [1.2], Step [0.6666666666666666], Loss: 0.0284
Epoch [1.2], Step [0.8333333333333334], Loss: 0.0137
Epoch [1.2], Step [1.0], Loss: 0.0194
Epoch [2.2], Step [0.16666666666666666], Loss: 0.0145
Epoch [2.2], Step [0.3333333333333333], Loss: 0.0386
Epoch [2.2], Step [0.5], Loss: 0.0832
Epoch [2.2], Step [0.6666666666666666], Loss: 0.0329
Epoch [2.2], Step [0.8333333333333334], Loss: 0.0070
Epoch [2.2], Step [1.0], Loss: 0.0476
Epoch [3.2], Step [0.16666666666666666], Loss: 0.0240
Epoch [3.2], Step [0.3333333333333333], Loss: 0.0107
Epoch [3.2], Step [0.5], L

In [2]:
model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        
    print(f"Test Accuracy of the model on the 10000 test images: {(100*correct / total)}")

Test Accuracy of the model on the 10000 test images: 98.51


In [3]:
torch.save(model.state_dict(), 'model.ckpt')